In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests

def fetch_article_list(params: dict) -> list[dict]:
    url = "https://m.land.naver.com/cluster/ajax/articleList"
    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://m.land.naver.com/",
        "Accept": "application/json, text/plain, */*",
    })
    r = s.get(url, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()

    # ── 안전 분기 ──
    body = data.get("body")  # ← 지금 케이스에서 존재
    if body is None:
        # 드물게 전체가 리스트/혹은 다른 래핑으로 올 수 있음
        if isinstance(data, list):
            items = data
        elif isinstance(data, dict):
            # articles가 최상위로 오는 변형 케이스
            items = data.get("articles", []) or data.get("list", [])
        else:
            items = []
    else:
        # body가 리스트 vs 딕셔너리 모두 처리
        if isinstance(body, list):
            items = body
        elif isinstance(body, dict):
            items = body.get("articles") or body.get("list") or []
        else:
            items = []

    return items, data.get("more"), data.get("page")

# 예시 파라미터
params = {
    "rletTpCd": "APT",            # APT/OPST/GM …
    "tradTpCd": "A1:B1:B2",       # A1=매매, B1=전세, B2=월세
    "z": "17",
    "lat": "37.4979",
    "lon": "127.0276",
    # 필요시: "cortarNo": "1168000000",
    "page": "1",
}

items, more, page = fetch_article_list(params)

# 필드가 자주 바뀌므로 키 존재 체크
for it in items:
    atcl_no = it.get("atclNo") or it.get("articleNo")
    name    = it.get("atclNm") or it.get("articleName")
    price   = it.get("prc")    or it.get("priceString")
    floor   = it.get("flrInfo") or it.get("floorInfo")
    realtor = it.get("rltrNm") or it.get("realtorName")
    print(atcl_no, name, price, floor, realtor)


In [3]:
import requests, math, time, random

S = requests.Session()
S.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://m.land.naver.com/",
    "Accept": "application/json, text/plain, */*",
})

params = {
    "view": "atcl",
    "rletTpCd": "APT",              # 아파트
    "tradTpCd": "A1:B1:B2",         # 매매/전세/월세
    "z": "17",
    "lat": "37.4979",
    "lon": "127.0276",
    "cortarNo": "1168000000",       # (선택) 강남구 예시
    # 지도 bbox를 넣으면 더 정확함:
    # "btm": "...", "lft": "...", "top": "...", "rgt": "..."
}




CLUSTER_URL = "https://m.land.naver.com/cluster/clusterList"
r = S.get(CLUSTER_URL, params=params, timeout=15)
r.raise_for_status()
cl = r.json()
# 클러스터에서 그룹(버블) 꺼내기
groups = (cl.get("data") or {}).get("ARTICLE") or []

# for i in range(10):
#     CLUSTER_URL = "https://m.land.naver.com/cluster/clusterList"
#     r2 = S.get(CLUSTER_URL, params=params, timeout=15)
#     r2.raise_for_status()
#     cl2 = r2.json()
#     groups.extend((cl2.get("data") or {}).get("ARTICLE") or [])


print(type(groups))
print("그룹 개수:", len(groups))

<class 'list'>
그룹 개수: 500


In [ ]:
ART_URL = "https://m.land.naver.com/cluster/ajax/articleList"
all_items = []

for g in groups[:2]:
 lgeo = g["lgeo"] # ← 그룹 식별자 (중요)
 count = int(g["count"])
 z2 = g["z"]
 lat2 = g["lat"]
 lon2 = g["lon"]

 pages = max(1, math.ceil(count / 20)) # 네이버가 20개 단위 반환

 for page in range(1, pages + 1):
  q = {
  "itemId": lgeo, # = lgeo
  "mapKey": "",
  "lgeo": lgeo,
  "showR0": "",
  "rletTpCd": params["rletTpCd"],
  "tradTpCd": params["tradTpCd"],
  "z": z2,
  "lat": lat2,
  "lon": lon2,
  "totCnt": count, # 그룹 총개수(중요)
  "cortarNo": params.get("cortarNo", ""),
  "page": str(page),
  }
  rr = S.get(ART_URL, params=q, timeout=15)
  rr.raise_for_status()
  try:
    data = rr.json()

    # 응답 래핑 표준화
    body = data.get("body")
    if isinstance(body, list):
      items = body
    elif isinstance(body, dict):
      items = body.get("articles") or body.get("list") or []
    else:
      items = data if isinstance(data, list) else data.get("articles", [])

    all_items.extend(items)
    time.sleep(random.uniform(0.8, 1.6))
  except : 
    continue

print("수집 매물 수:", len(all_items))
# 필드 예시 출력
for it in all_items[:5]:
  print(
    it.get("atclNo") or it.get("articleNo"),
    it.get("atclNm") or it.get("articleName"),
    it.get("prc") or it.get("priceString"),
    it.get("flrInfo") or it.get("floorInfo"),
    it.get("rltrNm") or it.get("realtorName"),
  )

In [8]:
if all_items:
    print("\n[수집된 매물 데이터 샘플 (최대 10개)]")
    for it in all_items[:10]:
        atcl_no = it.get("atclNo") or it.get("articleNo", "N/A")
        name = it.get("atclNm") or it.get("articleName", "N/A")
        price = it.get("prc") or it.get("priceString", "N/A")
        floor = it.get("flrInfo") or it.get("floorInfo", "N/A")
        realtor = it.get("rltrNm") or it.get("realtorName", "N/A")
        print(f"  - 매물번호: {atcl_no}, 이름: {name}, 가격: {price * 10000}, 층: {floor}")


[수집된 매물 데이터 샘플 (최대 10개)]
  - 매물번호: 2556748505, 이름: 아크로삼성, 가격: 1800000000, 층: 10/25
  - 매물번호: 2556296931, 이름: 아크로삼성, 가격: 2100000000, 층: 13/25
  - 매물번호: 2556296016, 이름: 아크로삼성, 가격: 5800000000, 층: 5/25
  - 매물번호: 2555322289, 이름: 아크로삼성, 가격: 1500000000, 층: 22/25
  - 매물번호: 2554576675, 이름: 아크로삼성, 가격: 3000000000, 층: 11/25
  - 매물번호: 2555148713, 이름: 아크로삼성, 가격: 8000000000, 층: 14/25
  - 매물번호: 2554889129, 이름: 아크로삼성, 가격: 2500000000, 층: 14/25
  - 매물번호: 2554584923, 이름: 아크로삼성, 가격: 300000000, 층: 10/25
  - 매물번호: 2554585467, 이름: 아크로삼성, 가격: 500000000, 층: 24/25
  - 매물번호: 2554576410, 이름: 아크로삼성, 가격: 4300000000, 층: 11/25


In [9]:
all_items[0]

{'atclNo': '2556748505',
 'cortarNo': '1168010500',
 'atclNm': '아크로삼성',
 'atclStatCd': 'R0',
 'rletTpCd': 'A01',
 'uprRletTpCd': 'A01',
 'rletTpNm': '아파트',
 'tradTpCd': 'B1',
 'tradTpNm': '전세',
 'vrfcTpCd': 'S_VR',
 'flrInfo': '10/25',
 'prc': 180000,
 'rentPrc': 0,
 'hanPrc': '18억',
 'spc1': '116',
 'spc2': '92',
 'direction': '남동향',
 'atclCfmYmd': '25.10.23.',
 'repImgUrl': '/20251023_269/1761213421579jRkSS_JPEG/2e45c81af3e1c504ac4c808a5ba4b4e8.JPG',
 'repImgTpCd': 'SITE',
 'repImgThumb': 'f130_98',
 'lat': 37.5196,
 'lng': 127.059276,
 'atclFetrDesc': '신축 아크로삼성92B첫입주전세',
 'tagList': ['2년이내', '역세권', '방세개'],
 'bildNm': '103동',
 'minute': 0,
 'sameAddrCnt': 7,
 'sameAddrDirectCnt': 0,
 'sameAddrHash': '24A01B1N528ebcdc9e8695ca40b4ccfc5c91a4278aa34966b07d4f282ed41bd399fc038e',
 'sameAddrMaxPrc': '20억',
 'sameAddrMinPrc': '18억',
 'cpid': 'NEONET',
 'cpNm': '부동산뱅크',
 'cpCnt': 2,
 'rltrNm': '차차라움청담부동산중개',
 'directTradYn': 'N',
 'minMviFee': 0,
 'maxMviFee': 0,
 'etRoomCnt': 0,
 'tradePrice